In [21]:
import os
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
## can we please make this a relative path
DATABASE_DIR = "C:\\Users\\Ewan\\Documents\\CodeForBoston\\SafeWater\\data\\swdis\\"

# Allow us to display all the columns in a dataframe
pd.options.display.max_columns = None

In [22]:
# Read in the data from the CSV files
def read_in_dataframes(root_dir):
    """Read in the SDWIS tables into dataframes and save them in a dictionary"""
    tables = ['ENFORCEMENT_ACTION',
              'GEOGRAPHIC_AREA',
              'LCR_SAMPLE',
              'LCR_SAMPLE_RESULT',
              'SERVICE_AREA',
              'TREATMENT',
              'VIOLATION',
              'VIOLATION_ENF_ASSOC',
              'WATER_SYSTEM',
              'WATER_SYSTEM_FACILITY']
    df = {}
    for table in tables:
        df[table] = pd.read_csv("{0}{1}{2}.csv".format(root_dir, os.path.sep, table),
                                index_col=False,
                                error_bad_lines=False)
    
    return df

df = read_in_dataframes(DATABASE_DIR)

b'Skipping line 2780: expected 7 fields, saw 8\nSkipping line 4943: expected 7 fields, saw 8\nSkipping line 6784: expected 7 fields, saw 8\nSkipping line 7030: expected 7 fields, saw 8\nSkipping line 9079: expected 7 fields, saw 8\nSkipping line 11087: expected 7 fields, saw 8\nSkipping line 12463: expected 7 fields, saw 8\nSkipping line 21912: expected 7 fields, saw 11\nSkipping line 25245: expected 7 fields, saw 8\nSkipping line 25462: expected 7 fields, saw 9\nSkipping line 25610: expected 7 fields, saw 8\nSkipping line 25790: expected 7 fields, saw 8\nSkipping line 26205: expected 7 fields, saw 8\nSkipping line 27561: expected 7 fields, saw 14\nSkipping line 28035: expected 7 fields, saw 8\nSkipping line 32665: expected 7 fields, saw 11\nSkipping line 35567: expected 7 fields, saw 9\nSkipping line 35568: expected 7 fields, saw 13\nSkipping line 35585: expected 7 fields, saw 10\nSkipping line 35640: expected 7 fields, saw 13\nSkipping line 35648: expected 7 fields, saw 9\nSkipping l

b'Skipping line 1151188: expected 7 fields, saw 9\nSkipping line 1178609: expected 7 fields, saw 8\nSkipping line 1178623: expected 7 fields, saw 8\nSkipping line 1178675: expected 7 fields, saw 11\n'
b'Skipping line 1184796: expected 7 fields, saw 11\nSkipping line 1195050: expected 7 fields, saw 9\nSkipping line 1219169: expected 7 fields, saw 17\nSkipping line 1224766: expected 7 fields, saw 9\nSkipping line 1270641: expected 7 fields, saw 14\nSkipping line 1270897: expected 7 fields, saw 8\nSkipping line 1270961: expected 7 fields, saw 8\nSkipping line 1271254: expected 7 fields, saw 8\nSkipping line 1271941: expected 7 fields, saw 8\nSkipping line 1272263: expected 7 fields, saw 10\nSkipping line 1273002: expected 7 fields, saw 8\nSkipping line 1273017: expected 7 fields, saw 8\nSkipping line 1274018: expected 7 fields, saw 8\nSkipping line 1274019: expected 7 fields, saw 8\nSkipping line 1274707: expected 7 fields, saw 9\nSkipping line 1274939: expected 7 fields, saw 8\nSkipping 

b'Skipping line 892593: expected 22 fields, saw 23\nSkipping line 892594: expected 22 fields, saw 23\nSkipping line 893543: expected 22 fields, saw 23\nSkipping line 893687: expected 22 fields, saw 23\nSkipping line 893688: expected 22 fields, saw 23\nSkipping line 893689: expected 22 fields, saw 23\nSkipping line 894044: expected 22 fields, saw 23\nSkipping line 894358: expected 22 fields, saw 24\nSkipping line 894368: expected 22 fields, saw 24\nSkipping line 894372: expected 22 fields, saw 23\nSkipping line 894379: expected 22 fields, saw 23\nSkipping line 894521: expected 22 fields, saw 23\nSkipping line 894689: expected 22 fields, saw 24\nSkipping line 895014: expected 22 fields, saw 24\n'
b'Skipping line 1183545: expected 22 fields, saw 23\n'
C:\Users\Ewan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (4,7,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [23]:
def prepare_initial_model_df(df, from_year=2018, num_years=5):
    """Create an initial dataframe which contains water system info on a yearly basis."""
    num_pws_ids = df.shape[0]
    
    # Create num_years copies of the input df stacked vertically
    model_df = df
    for ii in range(num_years-1):
        model_df = pd.concat((model_df, df))

    # Add the year column to the model df
    ones = np.ones(num_pws_ids)
    years_col = np.array([], dtype=int)
    for year in range(from_year, from_year - num_years, -1):
        years_col = np.concatenate((years_col, (year * ones).astype(int)))
    
    model_df['YEAR'] = years_col
    
    return model_df

In [24]:
def get_year_column(df, initial_col):
    """Add a YEAR column to a df containing the year"""
    df['DATETIME'] = pd.to_datetime(df[initial_col])
    df['YEAR'] = df['DATETIME'].apply(lambda x: x.year)
    
    return df

In [25]:
def get_yearly_num_violations(viol_df, year_min=None):
    """Calculate the number of violations in each year.
    
    From viol_df, calculate the number of health based and non-health based violations 
    on a per year per PWSID basis.
    
    Output a dataframe with columns for PWSID, YEAR, NUM_HEALTH_BASED_VIOLATIONS, 
    NUM_NON_HEALTH_BASED_VIOLATIONS and HEALTH_BASED_VIOLATION_THIS_YEAR which is indexed on PWSID and YEAR.
    The HEALTH_BASED_VIOLATION_THIS_YEAR is a categorical variable which takes a value of 1 or 0 depending
    on whether there was a health based violation that year.
    """
    viol_df = get_year_column(viol_df, 'COMPL_PER_BEGIN_DATE')
    
    # Remove entries from viol_df from before year_min
    if year_min is not None:
        viol_df = viol_df[viol_df['YEAR'] >= year_min]
        
    hb_viol_counts = viol_df.groupby(['PWSID', 'YEAR', 'IS_HEALTH_BASED_IND'], as_index=False).count()
    
    # Each column in hb_viol_counts just includes the number of violations. So just create an explicit column
    # for number of violations and drop all the other columns
    hb_viol_counts['NUM_VIOLATIONS'] = hb_viol_counts['COMPLIANCE_STATUS_CODE']
    hb_viol_counts = hb_viol_counts[['PWSID', 'YEAR', 'IS_HEALTH_BASED_IND', 'NUM_VIOLATIONS']]
    
    # Create seperate data frames for the health based and non-health based violations 
    non_hb_viol_counts = hb_viol_counts[hb_viol_counts['IS_HEALTH_BASED_IND'] == 'N']
    non_hb_viol_counts.drop('IS_HEALTH_BASED_IND', axis=1, inplace=True)
    non_hb_viol_counts.rename(columns={'NUM_VIOLATIONS': 'NUM_NON_HEALTH_BASED_VIOLATIONS'}, inplace=True)
    
    hb_viol_counts = hb_viol_counts[hb_viol_counts['IS_HEALTH_BASED_IND'] == 'Y']
    hb_viol_counts.drop('IS_HEALTH_BASED_IND', axis=1, inplace=True)
    hb_viol_counts.rename(columns={'NUM_VIOLATIONS': 'NUM_HEALTH_BASED_VIOLATIONS'}, inplace=True)
    
    # Merge the non health based and health based data frames to get a dataframe that
    # has one column for number of health based violations and one column for non
    # health based violations.
    viol_counts = hb_viol_counts.merge(non_hb_viol_counts, on=['PWSID', 'YEAR'], how='outer')
    viol_counts['NUM_HEALTH_BASED_VIOLATIONS'].fillna(0, inplace=True)
    viol_counts['NUM_NON_HEALTH_BASED_VIOLATIONS'].fillna(0, inplace=True)
    viol_counts['HEALTH_BASED_VIOLATION_THIS_YEAR'] = (viol_counts['NUM_HEALTH_BASED_VIOLATIONS'] > 0).astype(int)
    
    return viol_counts


In [26]:
def add_more_year_cols(df, num_years=5):
    """Add additional 'YEAR+n' columns to the dataframe.
    
    df must be a dataframe with a year column. Add 
    'YEAR+1', 'YEAR+2', ..., 'YEAR+num_years' columns to the dataframe"""
    for i in range(1, num_years+1):
        df['YEAR+{}'.format(i)] = df['YEAR'].apply(lambda year: year+i)
    
    return df

In [54]:
def add_dependent_variable_to_model(model_df, dep_var_df):
    """Merge the target/dependent variable into the model"""
    model_df = model_df.merge(dep_var_df, on=['PWSID', 'YEAR'], how='outer')
    model_df['HEALTH_BASED_VIOLATION_THIS_YEAR'].fillna(0, inplace=True)
    model_df = model_df[model_df['EPA_REGION'].notna()]
    
    return model_df 

In [58]:
def merge_cols_into_model(model_df, add_df, year_depth=5):
    """Merge the additional data from add_df into model_df.
    
    add_df contains some columns on a per year per PWSID basis. We want to add
    those columns to model_df. For each column in add_df we want to add historical
    data to model_df for year_depth years back. 
    
    e.g. if we have column 'COL1' in add_df and year_depth=3 then we want to add
    'COL1_PREVIOUS_YEAR', 'COL1_2_YEARS_AGO' and 'COL1_3_YEARS_AGO' to model_df.
    """
    # Get the list of columns to add. i.e. not the PWSID or YEAR columns
    columns_to_add = list(
        filter(lambda col: not col == 'PWSID' and not col.startswith('YEAR'),
               add_df.columns))
    for i in range(1, year_depth+1):
        cols_to_merge = columns_to_add + ['PWSID', 'YEAR+{}'.format(i)]
        model_df = model_df.merge(add_df[cols_to_merge], left_on=['PWSID', 'YEAR'], 
                                  right_on=['PWSID', 'YEAR+{}'.format(i)], how='outer')
        model_df.drop('YEAR+{}'.format(i), axis=1, inplace=True)
        
        for col in columns_to_add:
            model_df[col].fillna(0, inplace=True) 
        
        if i == 1:
            new_col_name = '{}_PREVIOUS_YEAR'
            
        else:
            years_ago = '{}_YEARS_AGO'.format(int(i))
            new_col_name = '{}_' + years_ago
        

        model_df.rename(columns={col: new_col_name.format(col) for col in columns_to_add},
                            inplace=True)
        
    model_df = model_df[model_df['YEAR'].notna()]
    
    return model_df
    

#sample_model_df = model_df[model_df.PWSID.isin(sample_grouped_viol_counts.PWSID)]
#display(sample_model_df)
#sample_model_df = sample_model_df.merge(sample_grouped_viol_counts, on=['PWSID', 'YEAR'], how='outer')
#sample_model_df['NUM_VIOLATIONS'] = sample_model_df['NUM_VIOLATIONS'].fillna(0)
#display(sample_model_df)

In [47]:
def get_yearly_num_enforcements(enf_df, year_min=None):
    """Calculate the number of enforcement actions in each year.
    
    From enf_df, calculate the number of enforcement actions 
    on a per year per PWSID basis.
    
    Output a dataframe with columns for PWSID, YEAR and NUM_ENFORCEMENTS 
    which is indexed on PWSID and YEAR.
    """
    enf_df = get_year_column(enf_df, 'ENFORCEMENT_DATE')
    
    # Remove entries from enf_df from before year_min
    if year_min is not None:
        enf_df = enf_df[enf_df['YEAR'] >= year_min]
        
    enf_counts = enf_df.groupby(['PWSID', 'YEAR'], as_index=False).count()
    
    # Each column in enf_counts just includes the number of enforcements. So just create an explicit column
    # for number of enforcements and drop all the other columns
    enf_counts['NUM_ENFORCEMENTS'] = enf_counts['ENFORCEMENT_ACTION_TYPE_CODE']
    enf_counts = enf_counts[['PWSID', 'YEAR', 'NUM_ENFORCEMENTS']]
    
    return enf_counts
    

In [56]:
def build_model_df(water_sys_df, viol_df, enf_df):
    """Build a dataframe that can be used to model the data."""
    model_df = prepare_initial_model_df(water_sys_df)
    
    # Create a df containing the number of health based and non-health based violations on
    # a per-year per-PWSID basis
    viol_counts = get_yearly_num_violations(viol_df, year_min=2009)
    viol_counts = add_more_year_cols(viol_counts, 5)
    print("Display table of number of violations")
    display(viol_counts)

    # Merge the dependent/target y variable to the model: 'HEALTH_BASED_VIOLATION_THIS_YEAR'
    model_df = add_dependent_variable_to_model(
        model_df, 
        viol_counts[['PWSID', 'YEAR', 'HEALTH_BASED_VIOLATION_THIS_YEAR']])
    print("display model_df after adding dependent variable")
    display(model_df)
    viol_counts.drop('HEALTH_BASED_VIOLATION_THIS_YEAR', axis=1, inplace=True)

    # Merge the num_violations columns in to the main model dataframe
    model_df = merge_cols_into_model(model_df, viol_counts, year_depth=5)
    res_model_df = model_df[['PWSID', 'YEAR', 'NUM_NON_HEALTH_BASED_VIOLATIONS_PREVIOUS_YEAR', 'NUM_NON_HEALTH_BASED_VIOLATIONS_2_YEARS_AGO']]
    print("Display the number of non health based violations over the last 2 years")
    display(res_model_df[res_model_df['NUM_NON_HEALTH_BASED_VIOLATIONS_PREVIOUS_YEAR'] > 0])
    display(res_model_df[res_model_df['NUM_NON_HEALTH_BASED_VIOLATIONS_2_YEARS_AGO'] > 0])

    # Create a df containing the number of enforcements on a per-year per-PWSID basis
    enf_counts = get_yearly_num_enforcements(enf_df, year_min=2009)
    enf_counts = add_more_year_cols(enf_counts, 5)
    print("display table of enforcement counts")
    display(enf_counts)

    # Merge the num_enforcements columns in to the main model dataframe
    model_df = merge_cols_into_model(model_df, enf_counts, year_depth=5)
    res_model_df = model_df[['PWSID', 'YEAR', 'NUM_ENFORCEMENTS_PREVIOUS_YEAR', 'NUM_ENFORCEMENTS_4_YEARS_AGO']]
    print("display number of enforcements last year and 4 years ago")
    display(res_model_df[res_model_df['NUM_ENFORCEMENTS_PREVIOUS_YEAR'] > 0])
    display(res_model_df[res_model_df['NUM_ENFORCEMENTS_4_YEARS_AGO'] > 0])
    
    return model_df


In [36]:
# Pick columns from the water system dataframe
water_system_df = df['WATER_SYSTEM'][['EPA_REGION', 'POPULATION_SERVED_COUNT', 'PRIMACY_TYPE', 'PRIMARY_SOURCE_CODE',
                                  'PWS_ACTIVITY_CODE', 'PWSID', 'PWS_TYPE_CODE', 'SERVICE_CONNECTIONS_COUNT',
                                  'STATE_CODE', 'ZIP_CODE']].copy()
display(water_system_df.head())

,EPA_REGION,POPULATION_SERVED_COUNT,PRIMACY_TYPE,PRIMARY_SOURCE_CODE,PWS_ACTIVITY_CODE,PWSID,PWS_TYPE_CODE,SERVICE_CONNECTIONS_COUNT,STATE_CODE,ZIP_CODE,YEAR
0,1,39552,Tribal,GU,A,010106001,CWS,143,CT,06339-3060,2018
1,1,41185,Tribal,GW,I,010109001,NTNCWS,1,NaN,NaN,2018
2,1,37860,Tribal,SWP,A,010109005,CWS,20,CT,06382,2018
3,1,84,Tribal,GW,A,010307001,CWS,33,MA,02535,2018
4,1,30,Tribal,GW,I,010502001,TNCWS,2,RI,02813,2018


,EPA_REGION,POPULATION_SERVED_COUNT,PRIMACY_TYPE,PRIMARY_SOURCE_CODE,PWS_ACTIVITY_CODE,PWSID,PWS_TYPE_CODE,SERVICE_CONNECTIONS_COUNT,STATE_CODE,ZIP_CODE,YEAR
421239,8,25,State,GW,A,WY5680261,TNCWS,1,WY,82520,2014
421240,8,25,State,GW,A,WY5680262,TNCWS,1,WY,82520,2014
421241,8,150,State,GW,A,WY5680263,TNCWS,9,WY,82801,2014
421242,8,77,State,SW,I,WY5689999,NTNCWS,77,WY,88888,2014
421243,8,450,State,GW,I,WY5690006,TNCWS,0,WY,82524,2014


In [37]:
# Pick columns from the violations data frame
violation_df = df['VIOLATION'][['COMPLIANCE_STATUS_CODE', 'COMPL_PER_BEGIN_DATE', 'COMPL_PER_END_DATE',
                                'CONTAMINANT_CODE', 'CORRECTIVE_ACTION_ID', 'FACILITY_ID',
                                'IS_HEALTH_BASED_IND', 'IS_MAJOR_VIOL_IND', 'LATEST_ENFORCEMENT_ID',
                                'ORIGINATOR_CODE', 'POPULATION_SERVED_COUNT', 'POP_CAT_5_CODE',
                                'PRIMACY_AGENCY_CODE', 'PRIMARY_SOURCE_CODE', 'PUBLIC_NOTIFICATION_TIER',
                                'PWSID', 'RTC_DATE', 'RTC_ENFORCEMENT_ID', 'RULE_CODE', 'RULE_FAMILY_CODE',
                                'RULE_GROUP_CODE', 'SEVERITY_IND_CNT', 'STATE_MCL', 'UNIT_OF_MEASURE',
                                'VIOLATION_CATEGORY_CODE', 'VIOLATION_CODE', 'VIOLATION_ID', 'VIOL_MEASURE']].copy()

violation_df.head()

,COMPLIANCE_STATUS_CODE,COMPL_PER_BEGIN_DATE,COMPL_PER_END_DATE,CONTAMINANT_CODE,CORRECTIVE_ACTION_ID,FACILITY_ID,IS_HEALTH_BASED_IND,IS_MAJOR_VIOL_IND,LATEST_ENFORCEMENT_ID,ORIGINATOR_CODE,POPULATION_SERVED_COUNT,POP_CAT_5_CODE,PRIMACY_AGENCY_CODE,PRIMARY_SOURCE_CODE,PUBLIC_NOTIFICATION_TIER,PWSID,RTC_DATE,RTC_ENFORCEMENT_ID,RULE_CODE,RULE_FAMILY_CODE,RULE_GROUP_CODE,SEVERITY_IND_CNT,STATE_MCL,UNIT_OF_MEASURE,VIOLATION_CATEGORY_CODE,VIOLATION_CODE,VIOLATION_ID,VIOL_MEASURE
0,R,01-JAN-16,31-DEC-16,2037,NaN,1,N,Y,0106001OX08031722932,R,39552,4,1,GU,3,10106001,03-AUG-17,0106001OX08031722932,320,320,300,NaN,NaN,NaN,MR,03,010600103203712016,NaN
1,R,01-JAN-16,31-DEC-16,2040,NaN,1,N,Y,0106001OX08031722931,R,39552,4,1,GU,3,10106001,03-AUG-17,0106001OX08031722931,320,320,300,NaN,NaN,NaN,MR,03,010600103204012016,NaN
2,R,01-JAN-16,31-DEC-16,2036,NaN,1,N,Y,0106001OX08031722930,R,39552,4,1,GU,3,10106001,03-AUG-17,0106001OX08031722930,320,320,300,NaN,NaN,NaN,MR,03,010600103203612016,NaN
3,R,01-JAN-16,31-DEC-16,2042,NaN,1,N,Y,0106001OX08031722929,R,39552,4,1,GU,3,10106001,03-AUG-17,0106001OX08031722929,320,320,300,NaN,NaN,NaN,MR,03,010600103204212016,NaN
4,R,01-JAN-16,31-DEC-16,2274,NaN,1,N,Y,0106001OX08031722928,R,39552,4,1,GU,3,10106001,03-AUG-17,0106001OX08031722928,320,320,300,NaN,NaN,NaN,MR,03,010600103227412016,NaN


In [38]:
# Pick columns from the enforcement action dataframe
enforce_df = df['ENFORCEMENT_ACTION'][['ENFORCEMENT_ACTION_TYPE_CODE', 'ENFORCEMENT_COMMENT_TEXT',
                                          'ENFORCEMENT_DATE', 'ENFORCEMENT_ID', 'ORIGINATOR_CODE','PWSID']].copy()
display(enforce_df.head())
display(enforce_df.count())

,ENFORCEMENT_ACTION_TYPE_CODE,ENFORCEMENT_COMMENT_TEXT,ENFORCEMENT_DATE,ENFORCEMENT_ID,ORIGINATOR_CODE,PWSID
0,EIF,Received the tribe's CCR which contained the P...,30-JUN-17,0106001IF06301722887,R,010106001
1,EOX,Received SOC results from lab.,03-AUG-17,0106001OX08031722922,R,010106001
2,EOX,Received SOC results from lab.,03-AUG-17,0106001OX08031722923,R,010106001
3,EOX,Received SOC results from lab.,03-AUG-17,0106001OX08031722924,R,010106001
4,EOX,Received SOC results from lab.,03-AUG-17,0106001OX08031722925,R,010106001


ENFORCEMENT_ACTION_TYPE_CODE    2324001
ENFORCEMENT_COMMENT_TEXT         345896
ENFORCEMENT_DATE                2324001
ENFORCEMENT_ID                  2324001
ORIGINATOR_CODE                 2323811
PWSID                           2324001
dtype: int64

In [61]:
# Create small samples from the water_system, violation and enforcement tables to test on
sample_water_sys_df = water_system_df.sample(20, random_state=42)
sample_viol_df = violation_df[violation_df.PWSID.isin(sample_water_sys_df.PWSID)]
sample_enf_df = enforce_df[enforce_df.PWSID.isin(sample_water_sys_df.PWSID)]

# Build a sample model dataframe
sample_model_df = build_model_df(sample_water_sys_df, sample_viol_df, sample_enf_df)

print("Here is the output samle model dataframe")
display(sample_model_df)
sample_model_df.to_csv("../data/sample_model.csv")

# @@TODO: Do one hot encoding for features

Display table of number of violations


C:\Users\Ewan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Ewan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Ewan\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

,PWSID,YEAR,NUM_HEALTH_BASED_VIOLATIONS,NUM_NON_HEALTH_BASED_VIOLATIONS,HEALTH_BASED_VIOLATION_THIS_YEAR,YEAR+1,YEAR+2,YEAR+3,YEAR+4,YEAR+5
0,MI2320283,2009,1.0,2,1,2010,2011,2012,2013,2014
1,FL3425056,2012,0.0,1,0,2013,2014,2015,2016,2017
2,FL3425056,2014,0.0,1,0,2015,2016,2017,2018,2019
3,FL3425056,2016,0.0,3,0,2017,2018,2019,2020,2021
4,FL3425056,2017,0.0,2,0,2018,2019,2020,2021,2022
5,MD1071087,2016,0.0,2,0,2017,2018,2019,2020,2021
6,MD1071087,2017,0.0,6,0,2018,2019,2020,2021,2022


display model_df after adding dependent variable


,EPA_REGION,POPULATION_SERVED_COUNT,PRIMACY_TYPE,PRIMARY_SOURCE_CODE,PWS_ACTIVITY_CODE,PWSID,PWS_TYPE_CODE,SERVICE_CONNECTIONS_COUNT,STATE_CODE,ZIP_CODE,YEAR,HEALTH_BASED_VIOLATION_THIS_YEAR
0,3,420.0,State,GW,I,VA6113197,NTNCWS,1,VA,22727,2018,0.0
1,2,135.0,State,GW,I,NY0019956,TNCWS,1,NaN,NaN,2018,0.0
2,5,25.0,State,GWP,A,IL3160549,TNCWS,1,IL,62946,2018,0.0
3,4,25.0,State,GW,I,FL3314129,TNCWS,2,FL,32957,2018,0.0
4,6,200.0,State,GW,I,AR8000379,TNCWS,1,AR,72855,2018,0.0
5,9,30.0,State,GW,N,CA3302044,TNCWS,16,NaN,NaN,2018,0.0
6,2,25.0,State,GW,I,NJ0818409,TNCWS,15,NJ,NaN,2018,0.0
7,4,25.0,State,GW,I,NC0382409,TNCWS,1,NC,28385,2018,0.0
8,6,50.0,State,GW,I,NM3597601,TNCWS,1,NM,87105,2018,0.0
9,5,35.0,State,GW,A,MI2320283,TNCWS,1,MI,48854,2018,0.0


...

display number of enforcements last year and 4 years ago


,PWSID,YEAR,NUM_ENFORCEMENTS_PREVIOUS_YEAR,NUM_ENFORCEMENTS_4_YEARS_AGO
14,MD1071087,2018.0,7.0,0.0
16,FL3425056,2018.0,4.0,0.0
34,MD1071087,2017.0,1.0,0.0
36,FL3425056,2017.0,1.0,0.0


,PWSID,YEAR,NUM_ENFORCEMENTS_PREVIOUS_YEAR,NUM_ENFORCEMENTS_4_YEARS_AGO
87,NC0382409,2014.0,0.0,4.0
89,MI2320283,2014.0,0.0,3.0


Here is the output samle model dataframe


,EPA_REGION,POPULATION_SERVED_COUNT,PRIMACY_TYPE,PRIMARY_SOURCE_CODE,PWS_ACTIVITY_CODE,PWSID,PWS_TYPE_CODE,SERVICE_CONNECTIONS_COUNT,STATE_CODE,ZIP_CODE,YEAR,HEALTH_BASED_VIOLATION_THIS_YEAR,NUM_HEALTH_BASED_VIOLATIONS_PREVIOUS_YEAR,NUM_NON_HEALTH_BASED_VIOLATIONS_PREVIOUS_YEAR,NUM_HEALTH_BASED_VIOLATIONS_2_YEARS_AGO,NUM_NON_HEALTH_BASED_VIOLATIONS_2_YEARS_AGO,NUM_HEALTH_BASED_VIOLATIONS_3_YEARS_AGO,NUM_NON_HEALTH_BASED_VIOLATIONS_3_YEARS_AGO,NUM_HEALTH_BASED_VIOLATIONS_4_YEARS_AGO,NUM_NON_HEALTH_BASED_VIOLATIONS_4_YEARS_AGO,NUM_HEALTH_BASED_VIOLATIONS_5_YEARS_AGO,NUM_NON_HEALTH_BASED_VIOLATIONS_5_YEARS_AGO,NUM_ENFORCEMENTS_PREVIOUS_YEAR,NUM_ENFORCEMENTS_2_YEARS_AGO,NUM_ENFORCEMENTS_3_YEARS_AGO,NUM_ENFORCEMENTS_4_YEARS_AGO,NUM_ENFORCEMENTS_5_YEARS_AGO
0,3,420.0,State,GW,I,VA6113197,NTNCWS,1,VA,22727,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,135.0,State,GW,I,NY0019956,TNCWS,1,NaN,NaN,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,25.0,State,GWP,A,IL3160549,TNCWS,1,IL,62946,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,25.0,State,GW,I,FL3314129,TNCWS,2,FL,32957,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,200.0,State,GW,I,AR8000379,TNCWS,1,AR,72855,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9,30.0,State,GW,N,CA3302044,TNCWS,16,NaN,NaN,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2,25.0,State,GW,I,NJ0818409,TNCWS,15,NJ,NaN,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4,25.0,State,GW,I,NC0382409,TNCWS,1,NC,28385,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,6,50.0,State,GW,I,NM3597601,TNCWS,1,NM,87105,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,5,35.0,State,GW,A,MI2320283,TNCWS,1,MI,48854,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Now build the real dataframe
model_df = build_model_df(water_system_df, violation_df, enforce_df)

print("Here is the full model dataframe in all it's glory!")
display(model_df)
model_df.to_csv("../data/simple_time_based_model.csv")